# Applications using Stacks

### Application: Balanced Parentheses using Stacks

<font size = "3">

- The idea is to determine whether an arithmetic operation is "balanced"

- For example, this is a balanced expression

```python
   (x*(5 + np.log(2)))
```

- This is an unbalanced expression
```python
    (5*(2 - 1)
```

- Each opening parenthesis must be followed by a corresponding closing parenthesis.

- This follows the same principle as a Stack:

<div style="text-align: center;">
  <img src="files/matching_parentheses.png" alt="Centered image" width = "350">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>



In [1]:
from datasci531 import Stack

In [2]:
def par_checker(symbol_string):
    s = Stack()
    for symbol in symbol_string:
        if symbol == "(":
            s.push(symbol)
        elif symbol == ")": # I modified this line from text's version
            if s.is_empty():
                return False
            else:
                s.pop()
    return s.is_empty()

In [3]:
print(par_checker("(x*(5 + np.log(2)))"))  # expected True
print(par_checker("(3 - ((x+y)(x-y) + 2))"))  # expected True
print(par_checker("(5*(2 - 1)"))  # expected False
print(par_checker(")("))  # expected False

True
True
False
False


<font size = "3">

- In Python, lists, tuples, and sets (dictionaries) are created using the delimiters `[]`, `()`, and `{}`

- We can nest these compound data types inside of each other, but the expression must be balanced.

- Balanced Python expressions:

```python
    x = {'name' : ['Mary', 'Jim', 'Gabe'], 'age' : [24, 30, 19]}

    y = [(11, 2, 13), (0, 7, 1)]
```

- Unbalanced expression:

```python
    z = [ {'state' : ('georgia', 'ohio',  'year' : (1788, 1803)}]
```

In [4]:
def balance_checker(symbol_string):
    s = Stack()
    for symbol in symbol_string:
        if symbol in "([{":
            s.push(symbol)
        elif symbol in ")]}":
            if s.is_empty():
                return False
            else:
                if not matches(s.pop(), symbol):
                    return False

    return s.is_empty()

def matches(sym_left, sym_right):
    all_lefts = "([{"
    all_rights = ")]}"
    return all_lefts.index(sym_left) == all_rights.index(sym_right)

In [5]:
print(balance_checker('{({([x + y][5])}(3,))}'))
print(balance_checker('[{()]'))
print(balance_checker("[(x, y, z)]"))

True
False
True


In [6]:
print(balance_checker("x = {'name' : ['Mary', 'Jim', 'Gabe'], 'age' : [24, 30, 19]}"))
print(balance_checker("y = [(11, 2, 13), (0, 7, 1)]"))
print(balance_checker("z = [ {'state' : ('georgia', 'ohio',  'year' : (1788, 1803)}]"))

True
True
False


### Application: Convert Base-10 to Binary

<font size = "3">

- The "divide-by-2" algorithm converts a number in Base 10 to binary by repeatedly dividing by 2 and keeping track of remainders.

- The binary representation is in the *opposite* order in which the remainders are calculated.

- A Stack is a natural data type to handle the remainders:

<div style="text-align: center;">
  <img src="files/remainders.png" alt="Centered image" width = "450">
  <figcaption><font size = "1"> Miller, Randum, Yasinovskyy (Problem Solving with Algorithms and Data Structures using Python)</figcaption>
</div>


In [7]:
import sys 

# sys.maxsize is the number of different memory locations that
# the processor can physically refer to
print(sys.maxsize)
print(2**63 - 1 == sys.maxsize)

9223372036854775807
True


In [8]:
def divide_by_2(decimal_num):
    rem_stack = Stack()
    for i in range(2**63 - 1): # replacement for while loop
        if decimal_num == 0:
            break 
        rem = decimal_num % 2
        rem_stack.push(rem)
        decimal_num = decimal_num // 2

    bin_string = ""
    for i in range(rem_stack.size()):
        bin_string += str(rem_stack.pop())
    
    return bin_string

In [9]:
print(divide_by_2(233))
print(divide_by_2(42))
print(divide_by_2(31))
print(divide_by_2(25))

11101001
101010
11111
11001


<font size = "3">

- We will generalize to handle other number bases besides binary.

- The most common number bases in computer science are binary, octal (base-8), and hexadecimal (base-16).

- Historical note: a ternary (base-3) computer was developed in 1958 at Moscow State University (the Setun)

- For base 16, we need additional digits besides the standard ten: 0, 1, 2, ..., 9, A, B, C, D, E, F

- We'll write the function so that it can convert for any base between 2 and 16.

In [10]:
def base_converter(decimal_num, base):
    digits = "0123456789ABCDEF"
    rem_stack = Stack()

    while decimal_num > 0:
        rem = decimal_num % base
        rem_stack.push(rem)
        decimal_num = decimal_num // base

    new_string = ""
    while not rem_stack.is_empty():
        new_string = new_string + digits[rem_stack.pop()]

    return new_string

print(base_converter(25, 2))
print(base_converter(25, 16))
print(base_converter(15, 16))
print(base_converter(26, 16))

11001
19
F
1A


In [11]:
# Hexadecimal (base-16) shows up in objects memory addresses
s = Stack()
print(s)

### Application: Postfix Notation

<font size = "3">

- How do you evaluate the expression $5\cdot 3 + 4 \cdot 3$? This is ambiguous, and can only be parsed by those who understand the standard conventions of **operator precedence**.

- We treat multiplication and division as operators with higher precedence than addition or subtraction. So we carry out multiplication first, and the expression is equivalent to $15 + 12$. The expression evaluates to 27.

- If we insert parentheses, we can disrupt operator precedence. The expression $5\cdot (3 + 4\cdot 3)$ indicates that the left-most multiplication should be carried out last. The expression evaluates to $5 \cdot 15 = 75$

- Computers must be told *exactly* what to compute. The first generation of computer scientists needed to guarantee unambiguous arithmetic. One approach is to put parentheses around everything:

\begin{align}
    ((5\cdot 3) + (4 \cdot 3))\\
    (5\cdot(3 + (4\cdot 3)))
\end{align}

- Another approach is to figure out how a computer can keep track of operator precedence while also adjusting these rules in the presence of parentheses (like we do)

- Yet another approach is to develop a new, unambiguous notation for arithmetic. One of these is **postfix** notation (also called reverse Polish notation). Here are the two expressions in the new notation:

\begin{align*}
    &5\ \  3\ \cdot\ 4\ \  3\ \cdot\ +\qquad \textrm{evaluates to 27}\\
    &5\ \ 3\ \ 4\ \ 3\ \cdot\ +\ \cdot\qquad \textrm{evaluates to 75}
\end{align*}

- Expression is evaluated left-to-right and uses a Stack.

In [12]:
def postfix_eval(postfix_expr):
    operand_stack = Stack()
    token_list = postfix_expr.split()

    for token in token_list:
        if token in "0123456789":
            operand_stack.push(int(token))
        else:
            operand2 = operand_stack.pop()
            operand1 = operand_stack.pop()
            result = do_math(token, operand1, operand2)
            operand_stack.push(result)
    return operand_stack.pop()


def do_math(op, op1, op2):
    if op == "*":
        return op1 * op2
    elif op == "/":
        return op1 / op2
    elif op == "+":
        return op1 + op2
    else:
        return op1 - op2

In [13]:
# Need white space in between operators and operands 
# for .split() to work correctly in function
my_str = "5 3 * 4 3 *+"

print(my_str.split())

my_str = "5 3 * 4 3 * +"
print(my_str.split())

['5', '3', '*', '4', '3', '*+']
['5', '3', '*', '4', '3', '*', '+']


In [14]:
print(postfix_eval("5 3 * 4 3 * +"))
print(postfix_eval("5 3 4 3 * + *"))
print(postfix_eval("7 8 + 3 2 + /"))

27
75
3.0


<font size = "3">

- We can convert a standard arithmetic expression (in **infix** notation) into postfix notation by using a Stack as well.

- The algorithm is as follows:

    1. Create empty stack `op_stack` for operators. Create empty list for output

    2. Convert the input string to a list of "tokens" (characters)

    3. Loop over token list

        - If the token is an operand (variable or number) append it to end of output list.

        - If token is `'('` push it on `op_stack`.

        - If token is `')'`, pop `op_stack` until a `'('` is removed, appending each operator to output list

        - If token is `*`, `/`, `+`, `-` check for operators of higher precedence on top of stack and popping them and appending them to output list. Then add token to `op_stack`.

    4. After all tokens have been processed, pop any operators on `op_stack` until the stack is emptied, appending them to the output list.

In [15]:
def infix_to_postfix(infix_expr):
    prec = {}
    prec["*"] = 3
    prec["/"] = 3
    prec["+"] = 2
    prec["-"] = 2
    prec["("] = 1
    op_stack = Stack()
    postfix_list = []
    token_list = infix_expr.split() # need white space in between every symbol for this to work

    for token in token_list:
        if token in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" or token in "0123456789":
            postfix_list.append(token)
        elif token == "(":
            op_stack.push(token)
        elif token == ")":
            top_token = op_stack.pop()
            while top_token != "(":
                postfix_list.append(top_token)
                top_token = op_stack.pop()
        else:
            while (not op_stack.is_empty()) and (prec[op_stack.peek()] >= prec[token]):
                postfix_list.append(op_stack.pop())
            op_stack.push(token)

    while not op_stack.is_empty():
        postfix_list.append(op_stack.pop())

    return " ".join(postfix_list)


In [16]:
print(infix_to_postfix("1 + ( 5 * 3 + 2 )"), '\n')

print(infix_to_postfix("1 + ( 5 + 3 * 2 )"))

1 5 3 * 2 + + 

1 5 3 2 * + +


In [17]:
print(infix_to_postfix("A * B + C * D"))
print(infix_to_postfix("( A + B ) * C - ( D - E ) * ( F + G )"))

A B * C D * +
A B + C * D E - F G + * -
